In [1]:
import random
import numpy as np
import codecs
from io import open

In [2]:
batch_size = 64       # Batch size for training.
epochs = 5            # Number of epochs to train for.
latent_dim = 256      # Latent dimensionality of the encoding space.
num_samples = 10000   # Number of samples to train on.
data_path = 'data/fra.txt' # Path to the data txt file on disk.

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [4]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

('Number of samples:', 10000)
('Number of unique input tokens:', 71)
('Number of unique output tokens:', 94)
('Max sequence length for inputs:', 16)
('Max sequence length for outputs:', 59)


In [5]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [6]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [7]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

#### Definition of the model architecture

In [8]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, GRUCell, RNN, Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers import RMSprop, adam

Using TensorFlow backend.
/Applications/anaconda/lib/python2.7/site-packages/cffi/model.py:532: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


#### I have made the following improvements to the architecture 
1. I have replaced the LSTM layer in the encoder and decoder with GRU layers respectively. The GRU layer introduced by Cho et. al. 2014 is much more simple and effective and immediately leads to a reduction in validation loss.
2. I have generalized the architecture by constructing deep encoder and deep decoder where the depth of the GRU layers in both of them is taken as an input. 
3. I have introduced an additional hidden layer followed by a dropout layer for processing of the output from the decoder GRU. 
All these steps have resulted in a drastic reduction in validation loss (final value is 0.4698 for depth=3) keeping the number of epochs fixed at 5.

In [9]:
depth=3 # depth of encoder and decoder
layers = [latent_dim for x in range(depth)]

encoder_inputs = Input(shape=(None, num_encoder_tokens))
# Create a list of GRU Cells, these are then concatenated into a single layer
# with the RNN layer.
encoder_cells = []
for hidden_neurons in layers:
    encoder_cells.append(GRUCell(hidden_neurons))
encoder = RNN(encoder_cells, return_state=True)
encoder_outputs_and_states = encoder(encoder_inputs)

# Discard encoder outputs and only keep the states as before (in the minimal implementation)
encoder_states = encoder_outputs_and_states[1:]
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Again create a list of GRU cells for the decoder which will then be concatenated with the RNN layer
decoder_cells = []
for hidden_neurons in layers:
    decoder_cells.append(GRUCell(hidden_neurons))
decoder = RNN(decoder_cells, return_sequences=True, return_state=True)

# Set the initial state of the decoder to be the ouput state of the encoder.

decoder_outputs_and_states = decoder(decoder_inputs, initial_state=encoder_states)

# Only select the output of the decoder (the states will be used for inference)
decoder_outputs = decoder_outputs_and_states[0]
#Introduce a hidden layer followed by a Dropout layer before the final Dense layer giving the outputs
decoder_hidden = Dense(256, activation='relu')
decoder_outputs = decoder_hidden(decoder_outputs)
decoder_outputs = Dropout(rate = 0.1)(decoder_outputs)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

In [10]:
model.compile(optimizer=adam(), loss='categorical_crossentropy')

#### End of Definition of the model architecture

In [11]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[EarlyStopping(patience=3, verbose=1), ReduceLROnPlateau(patience=1, min_lr=10e-6, verbose=1)],
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 137s 17ms/step - loss: 0.8936 - val_loss: 0.7915

In [12]:
model.save('my_trained_model.h5')

/Applications/anaconda/lib/python2.7/site-packages/keras/engine/network.py:872: UserWarning: Layer rnn_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'rnn_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'rnn_1/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'rnn_1/while/Exit_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [13]:
model = load_model('my_trained_model.h5')

### Inference mode (sampling).

1. Encode input and retrieve initial decoder state
2. Run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token
3. Repeat with the current target token and current states

In [14]:
def decode_sequence(input_seq, encoder_model, decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # The states must be a list
    if not isinstance(states_value, list):
        states_value = [states_value]


    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        outputs_and_states = decoder_model.predict(
            [target_seq] + states_value)
    
        output_tokens = outputs_and_states[0]
        states = outputs_and_states[1:]
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = states

    return decoded_sentence

#### Sampling Model Definition of the model architecture

In [15]:
#Build the corresponding decoder model which will be called iteratively
encoder_model = Model(encoder_inputs, encoder_states)

decoder_states_inputs = []

for hidden_neurons in layers[::-1]:
    decoder_states_inputs.append(Input(shape=(hidden_neurons,)))

decoder_outputs_and_states = decoder(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_outputs = decoder_outputs_and_states[0]
decoder_states = decoder_outputs_and_states[1:]

decoder_outputs = decoder_hidden(decoder_outputs)
decoder_outputs = Dropout(rate = 0.1)(decoder_outputs)
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

#### End of Sampling Model Definition of the model architecture

In [16]:
test_texts = ['Thanks.', 'Hello.', 'How are you?'] + [input_texts[i] for i in random.sample(range(len(input_texts)), 10)]

In [17]:
def encode_texts_to_1hot_seq(input_texts):
    input_seq = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for i, text in enumerate(input_texts):
        for t, char in enumerate(text):
            input_seq[i, t, input_token_index[char]] = 1.
    return input_seq

In [18]:
for test_text in test_texts:
    # Take one sequence (part of the training set) for trying out decoding.
    input_seq = encode_texts_to_1hot_seq([test_text])
    decoded_sentence = decode_sequence(input_seq, encoder_model, decoder_model)
    print('-')
    print('Input sentence:', test_text)
    print('Decoded sentence:', decoded_sentence)

-
('Input sentence:', 'Thanks.')
('Decoded sentence:', u'Ne te plais pas !\n')
-
('Input sentence:', 'Hello.')
('Decoded sentence:', u'Ne le plaisse pas.\n')
-
('Input sentence:', 'How are you?')
('Decoded sentence:', u'Est-ce que je partir pas ?\n')
-
('Input sentence:', u'We need rules.')
('Decoded sentence:', u'Nous sommes sauves.\n')
-
('Input sentence:', u'I assume so.')
('Decoded sentence:', u"Je l'ai pas de l'air train de maintenant.\n")
-
('Input sentence:', u'I hate politics.')
('Decoded sentence:', u'Je me suis pas four.\n')
-
('Input sentence:', u'He gazed at me.')
('Decoded sentence:', u'Il est trop venu.\n')
-
('Input sentence:', u'I want you.')
('Decoded sentence:', u"Je l'ai rendu.\n")
-
('Input sentence:', u'I got expelled.')
('Decoded sentence:', u'Je me suis pas four.\n')
-
('Input sentence:', u'Help me!')
('Decoded sentence:', u'Prends !\n')
-
('Input sentence:', u"I've no idea.")
('Decoded sentence:', u"Je l'ai pas de l'air train de maintenant.\n")
-
('Input sentenc